# Multilabel BERT Experiments

In this notebook we do some first experiments with BERT: we finetune a BERT model+classifier on each of our datasets separately and compute the accuracy of the resulting classifier on the test data.

For these experiments we use the `pytorch_transformers` package. It contains a variety of neural network architectures for transfer learning and pretrained models, including BERT and XLNET.

Two different BERT models are relevant for our experiments: 

- BERT-base-uncased: a relatively small BERT model that should already give reasonable results,
- BERT-large-uncased: a larger model for real state-of-the-art results.

In [1]:
PREFIXES = ["title9_but", "title9_because", "title9_so", "eatingmeat4_so", "eatingmeat4_because", "eatingmeat4_but"]  #"voting_so_automl"
BERT_MODEL = 'bert-base-uncased'
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
GRADIENT_ACCUMULATION_STEPS = 1 if "base" in BERT_MODEL else 8
MAX_SEQ_LENGTH = 100

## Data

We use the same data as for all our previous experiments. Here we load the training, development and test data for a particular prompt.

In [2]:
import sys
sys.path.append('../')

import ndjson
import glob
import numpy as np
from collections import Counter

from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary

data = []
for prefix in PREFIXES:
    data_file = f"../data/interim/{prefix}_withprompt.ndjson"

    with open(data_file) as i:
        new_data = ndjson.load(i)
        for item in new_data:
            item["labels"] = [prefix + "_" + l for l in item["labels"] if len(l) > 1]
            item["label"] = item["labels"]

        data.extend(new_data)

label2idx = create_label_vocabulary(data)
print(label2idx)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]
data_items = np.array(preprocess(data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH))

labels = Counter()
for item in data:
    labels.update(item["label"])
    
lc = Counter()
for item in data_items:
    for j, v in enumerate(item.label_ids):
        if v == 1:
            lc.update([j])
        
print(labels)
print(lc)

I0406 12:43:14.593649 139997413660480 file_utils.py:41] PyTorch version 1.2.0+cu92 available.
I0406 12:43:15.604099 139997413660480 file_utils.py:57] TensorFlow version 2.1.0 available.


{'title9_but_Affected_More_Men': 0, 'title9_but_Cuts_to_mens_sports': 1, 'title9_but_Cuts_to_WSD': 2, 'title9_but_Cuts_to_WSD,': 3, 'title9_but_EMU_compliant': 4, 'title9_but_Miscellaneous': 5, 'title9_but_Limit_woms_opps': 6, 'title9_but_Court_Decision': 7, 'title9_but_Judge_reinstate': 8, 'title9_but_Teams_reinstated': 9, 'title9_but_Women_sued': 10, 'title9_but_Kept_other_ones': 11, 'title9_but_Did_not_cut_t_and_s': 12, 'title9_but_EMU_viol_TitleIX': 13, 'title9_but_EMU_cut_less_men': 14, 'title9_but_Fem_athletes_mad': 15, 'title9_because_Fin_Trouble': 16, 'title9_because_Lack_of_funds': 17, 'title9_because_To_Save_Money': 18, 'title9_because_Miscellaneous': 19, 'title9_because_Overspending': 20, 'title9_so_Ath_claim_TitleIX': 21, 'title9_so_Miscellaneous': 22, 'title9_so_Ath_fought': 23, 'title9_so_Teams_got_reinstated': 24, 'title9_so_Ath_sued': 25, 'title9_so_Court_EMU_viol': 26, 'title9_so_Judge_said_reinstate': 27, 'title9_so_Reduce_costs': 28, 'title9_so_Ath_sued_TitleIX': 29,

I0406 12:43:16.310533 139997413660480 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Counter({'eatingmeat4_but_Feedback_13': 358, 'eatingmeat4_so_Feedback_9': 309, 'eatingmeat4_because_Feedback_1': 282, 'title9_because_Overspending': 276, 'eatingmeat4_so_Feedback_10': 241, 'title9_because_To_Save_Money': 224, 'title9_so_Reduce_costs': 209, 'eatingmeat4_because_Feedback_5': 205, 'title9_so_Ath_sued_TitleIX': 179, 'title9_because_Fin_Trouble': 172, 'eatingmeat4_so_Feedback_2': 148, 'title9_but_Miscellaneous': 124, 'title9_but_Teams_reinstated': 116, 'title9_so_Miscellaneous': 113, 'title9_but_Women_sued': 107, 'title9_so_Ath_sued': 102, 'eatingmeat4_but_Feedback_9': 101, 'title9_but_Cuts_to_WSD,': 92, 'eatingmeat4_but_Feedback_7': 80, 'title9_because_Lack_of_funds': 57, 'title9_because_Miscellaneous': 57, 'title9_but_EMU_viol_TitleIX': 56, 'eatingmeat4_but_Feedback_3': 56, 'title9_but_Court_Decision': 49, 'title9_but_Cuts_to_mens_sports': 48, 'eatingmeat4_so_Feedback_8': 48, 'title9_but_Judge_reinstate': 45, 'eatingmeat4_because_Feedback_7': 43, 'title9_but_Affected_More

## Model

We load the pretrained model and put it on a GPU if one is available. We also put the model in "training" mode, so that we can correctly update its internal parameters on the basis of our data sets.

In [3]:
def evaluate_output(all_correct, all_predicted):
    correct = 0
    at_least_one = 0
    fp, fn, tp, tn = 0, 0, 0, 0
    for c, p in zip(all_correct, all_predicted):
        if sum(c == p) == len(c):
            correct +=1

        for ci, pi in zip(c, p):
            if pi == 1 and ci == 1:
                at_least_one += 1
                break

        for ci, pi in zip(c, p):
            if pi == 1 and ci == 1:
                tp += 1
                same = 1
            elif pi == 1 and ci == 0:
                fp += 1
            elif pi == 0 and ci == 1:
                fn += 1
            else:
                tn += 1
                same =1

    precision = tp/(tp+fp) if tp+fp > 0 else 0
    recall = tp/(tp+fn) if tp+fn > 0 else 0
    fscore = 2*precision*recall/(precision+recall) if (precision+recall) > 0 else 0
    print("Data size:", len(all_predicted))
    print("P:", tp, "/", tp+fp, "=", precision)
    print("R:", tp, "/", tp+fn, "=", recall)
    print("F:", fscore)
    print("A:", correct/len(all_correct))
    print("AL1:", at_least_one/len(all_correct))

In [4]:
import torch

from quillnlp.models.bert.train import train, evaluate
from quillnlp.models.bert.models import get_multilabel_bert_classifier

from quillnlp.models.bert.preprocessing import get_data_loader
from sklearn.model_selection import KFold

import random
random.shuffle(data_items)

kf = KFold(n_splits=5, shuffle=True, random_state=1)
all_correct, all_predicted = [], []
all_test_data = []
for train_idx, test_idx in kf.split(data_items):

    train_and_dev_data, test_data = data_items[train_idx], data_items[test_idx]    
    cutoff = int(len(train_and_dev_data)/6*5)
    train_data = train_and_dev_data[:cutoff]
    dev_data = train_and_dev_data[cutoff:]
    
    print("Train size:", len(train_data))        
    print("Dev size:", len(dev_data))        
            
    train_dataloader = get_data_loader(train_data, BATCH_SIZE)
    dev_dataloader = get_data_loader(dev_data, BATCH_SIZE, shuffle=False)
    test_dataloader = get_data_loader(test_data, BATCH_SIZE, shuffle=False)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), device=device)
    output_model_file = train(model, train_dataloader, dev_dataloader, 
                              BATCH_SIZE, GRADIENT_ACCUMULATION_STEPS, 
                              device, num_train_epochs=100)
    
    print("Loading model from", output_model_file)
    device="cpu"

    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), model_file=output_model_file, device=device)
    model.eval()
    
    _, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)
    evaluate_output(test_correct, test_predicted)
    all_correct.extend(test_correct)
    all_predicted.extend(test_predicted)
    all_test_data.extend(test_data)

Train size: 2956
Dev size: 592


I0406 12:43:18.506703 139997413660480 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0406 12:43:18.508011 139997413660480 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd


Loss history: []
Dev loss: 0.49955145088402003


Epoch:   1%|          | 1/100 [00:40<1:07:38, 41.00s/it]


Loss history: [0.49955145088402003]
Dev loss: 0.3425517323854807


...




Epoch:  70%|███████   | 70/100 [48:37<20:41, 41.37s/it]


Loss history: [0.5213218041368433, 0.3512672838327047, 0.22659491606660792, 0.15170020992691452, 0.11356990703859844, 0.09469581637028102, 0.08492339019839829, 0.07952852808945887, 0.07629744426624195, 0.07419083468817375, 0.0725745868843955, 0.07107165012810682, 0.06928105591922193, 0.06736977509147413, 0.06515877099858748, 0.06267403186978521, 0.05988547669069187, 0.05735226267495671, 0.0545267711418706, 0.05180915575977918, 0.049151497213421644, 0.04689121790028907, 0.04473428246942726, 0.04255887372670947, 0.04059563582209316, 0.038926143048180116, 0.037120581680052986, 0.035901194539021804, 0.03453905779767681, 0.03356715910942168, 0.032434062327484824, 0.031215171958949114, 0.030425874919101998, 0.029301784982955135, 0.02863636497106101, 0.027768823368525184, 0.027125793635039717, 0.026067637888765014, 0.02596252937675328, 0.02543608532161326, 0.024767381466321042, 0.024453865213168634, 0.024056918063276523, 0.023693673740569, 0.02324753919163266, 0.022897354855730728, 0.0228723


Loss history: [0.5213218041368433, 0.3512672838327047, 0.22659491606660792, 0.15170020992691452, 0.11356990703859844, 0.09469581637028102, 0.08492339019839829, 0.07952852808945887, 0.07629744426624195, 0.07419083468817375, 0.0725745868843955, 0.07107165012810682, 0.06928105591922193, 0.06736977509147413, 0.06515877099858748, 0.06267403186978521, 0.05988547669069187, 0.05735226267495671, 0.0545267711418706, 0.05180915575977918, 0.049151497213421644, 0.04689121790028907, 0.04473428246942726, 0.04255887372670947, 0.04059563582209316, 0.038926143048180116, 0.037120581680052986, 0.035901194539021804, 0.03453905779767681, 0.03356715910942168, 0.032434062327484824, 0.031215171958949114, 0.030425874919101998, 0.029301784982955135, 0.02863636497106101, 0.027768823368525184, 0.027125793635039717, 0.026067637888765014, 0.02596252937675328, 0.02543608532161326, 0.024767381466321042, 0.024453865213168634, 0.024056918063276523, 0.023693673740569, 0.02324753919163266, 0.022897354855730728, 0.0228723

I0406 17:13:57.230393 139997413660480 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0406 17:13:57.231432 139997413660480 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd


Data size: 887
P: 730 / 833 = 0.8763505402160864
R: 730 / 949 = 0.7692307692307693
F: 0.8193041526374859
A: 0.7531003382187148
AL1: 0.7936865839909808


## Evaluation

In [5]:
evaluate_output(all_correct, all_predicted)

Data size: 4435
P: 3723 / 4272 = 0.8714887640449438
R: 3723 / 4763 = 0.7816502204492967
F: 0.8241283895960154
A: 0.7627959413754227
AL1: 0.8011273957158963


In [6]:
scores = {}
for item, predicted, correct in zip(all_test_data, all_predicted, all_correct):
    correct_labels = [idx2label[i] for i, l in enumerate(correct) if l == 1]
    predicted_labels = [idx2label[i] for i, l in enumerate(predicted) if l == 1]
    print("{}#{}#{}".format(item.text, ";".join(correct_labels), ";".join(predicted_labels)))    
        
    for label in predicted_labels + correct_labels:
        if label not in scores:
            scores[label] = {"tp": 0, "fp": 0, "fn": 0, "support": 0}
    
    for label in predicted_labels:
        if label in correct_labels:
            scores[label]["tp"] += 1
        else:
            scores[label]["fp"] += 1

    for label in correct_labels:
        scores[label]["support"] += 1
        if label not in predicted_labels:
            scores[label]["fn"] += 1
            
for label in scores:
    lp = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fp"]) if scores[label]["tp"] + scores[label]["fp"] > 0 else 0
    lr = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fn"]) if scores[label]["tp"] + scores[label]["fn"] > 0 else 0
    lf = 2*lp*lr/(lp+lr) if lp+lr > 0 else 0
    
    print(label, lp, lr, lf, scores[label]["support"])

Methane from cow burps harms the environment, but some people still want to eat regular beef.#eatingmeat4_but_Feedback_7#eatingmeat4_but_Feedback_7
Methane from cow burps harms the environment, so Impossible foods wants to make non-meat food items that taste like meat in order to stop the raising of animals for food purposes.#eatingmeat4_so_Feedback_2#eatingmeat4_so_Feedback_2
Eastern Michigan University cut women's tennis and softball, but it was found to be in violation of Title IX.#title9_but_EMU_viol_TitleIX#title9_but_EMU_viol_TitleIX
Methane from cow burps harms the environment because each cow belches approximately 50 gallons of methane per year which contributes to air pollution as well as global warming.#eatingmeat4_because_Feedback_5#eatingmeat4_because_Feedback_5
Eastern Michigan University cut women's tennis and softball, so a student athlete who had a softball scholarship teamed up with someone to sue the university for violating Title IX.#title9_so_Ath_sued_TitleIX#title9

Eastern Michigan University cut women's tennis and softball, but they were forced to reinstate these sports after being successfully sued for being in violation of Title IX.#title9_but_Teams_reinstated;title9_but_Women_sued#title9_but_Teams_reinstated;title9_but_Women_sued
Eastern Michigan University cut women's tennis and softball because because wanted to save money#title9_because_To_Save_Money#title9_because_To_Save_Money
Methane from cow burps harms the environment because each day they burp out 30-50 gallons of the greenhouse gas methane which damages the ozone layer.#eatingmeat4_because_Feedback_9#eatingmeat4_because_Feedback_5
Methane from cow burps harms the environment because it increases the earth's temperature called Global Warming.#eatingmeat4_because_Feedback_1#eatingmeat4_because_Feedback_1
Methane from cow burps harms the environment, so scientists have been testing different diets to reduce the methane from cow burps#eatingmeat4_so_Feedback_3#
Eastern Michigan Universi

Eastern Michigan University cut women's tennis and softball, so it would be able to reduce it's deficit spending#title9_so_Reduce_costs#title9_so_Reduce_costs
Eastern Michigan University cut women's tennis and softball because because they had to find a way to save money because they were way over budget.#title9_because_To_Save_Money;title9_because_Overspending#title9_because_To_Save_Money;title9_because_Overspending
Methane from cow burps harms the environment because Not everyone.#eatingmeat4_because_Feedback_8#eatingmeat4_because_Feedback_8
Methane from cow burps harms the environment, but we can consume impossible foods to lower the emissions and dependency on cows.#eatingmeat4_but_Feedback_5#
Methane from cow burps harms the environment because methane gas causes Earth's temperature to rise in a process called global warming.#eatingmeat4_because_Feedback_1#eatingmeat4_because_Feedback_1
Methane from cow burps harms the environment, so we need to change our habits and how we handle

Methane from cow burps harms the environment, so by either eating less meat, or by having cows eat less gassy food, we can lower the amount of methane that's put into the atmosphere.#eatingmeat4_so_Feedback_10#eatingmeat4_so_Feedback_10
Eastern Michigan University cut women's tennis and softball, but they argued the cuts would effect more men than women.#title9_but_Affected_More_Men#title9_but_Affected_More_Men
Eastern Michigan University cut women's tennis and softball because because it needed to make budget cuts as it was deeply in debt.#title9_because_To_Save_Money;title9_because_Overspending#title9_because_To_Save_Money;title9_because_Overspending
Eastern Michigan University cut women's tennis and softball, but later reinstated them after a judge ruled that they must do so.#title9_but_Teams_reinstated#title9_but_Teams_reinstated
Eastern Michigan University cut women's tennis and softball because because it claims it did not have enough money in the budget.#title9_because_Lack_of_f

Methane from cow burps harms the environment, but to address this problem, the company Impossible Foods wants to end all animal agriculture by 2035#eatingmeat4_but_Feedback_8#eatingmeat4_but_Feedback_8
Methane from cow burps harms the environment, so the current agricultural practices of the world should be changed.#eatingmeat4_so_Feedback_9#eatingmeat4_so_Feedback_9
Methane from cow burps harms the environment because the gases released contain methane which is harmful to the environment.#eatingmeat4_because_Feedback_5#eatingmeat4_because_Feedback_5
Methane from cow burps harms the environment, so another solution lies in what farmers feed cows#eatingmeat4_so_Feedback_10#
Eastern Michigan University cut women's tennis and softball because because of costs to the university.#title9_because_Overspending#title9_because_Fin_Trouble
Methane from cow burps harms the environment, so the need to change what farmers feed their cows#eatingmeat4_so_Feedback_10#eatingmeat4_so_Feedback_10
Eastern 

In [7]:
print(len([p for p in all_predicted if sum(p) == 0]), "/", len(all_predicted))

389 / 4435
